## Re-Working Clavis 

In [1]:
## This is Clavis
## Clavis is a Keyword Expansion and Categorization Tool for Digital Marketing & URL Generation

## For example if you have a keyword like "best shoes" and you want to expand it to "best shoes for cycling" and "best shoes for running"
## as well as categorize whether it is "Male", "Female", "Unisex" or "Kids" you can do that with Clavis
## Also identify whether a keyword is Generic or Branded (Generic = "best shoes", Branded = "Nike shoes")
## also it identifies the Topic of the keyword ("Accessories", "Cycling Jackets", "Mountain Biking Jackets", etc..)
## lastly it will also identify which ones should be "Deleted" and which ones should be "Kept"

## The input to this system are two different CSV Files:

## Input 1: Keywords.csv
## Keywords File: This is a CSV file with the following columns: Keywords
## Keywords File Example:
## Keywords: Best shoes, running shoes, etc..

## Input 2: Categories.csv
## Categories File: This is a CSV with the following columns: Category, Search For, Return
## Categories File Example:
## Category | Search For | Return
## Gender | | Unisex --> If Search For is empty, return Unisex
## Gender | man | Men
## Gender | woman | Women
## Gender | lady | Women
## Gender | boy | Kids
## Branded_generic | | Generic --> If Search For is empty, return Generic
## Branded_generic | nike | Branded
## Branded_generic | adidas | Branded
## Branded_generic | puma | Branded
## Branded_generic | reebok | Branded
## Topic | | None --> If Search For is empty, return None
## Topic | cleats | accessories
## topic | vest | Gilet
## Topic | jacket*cycling*mountain*jacket | mountain biking jackets --> asterisk is a wildcard character, it can be used to match multiple words

## Output:
## Output is a CSV file with the following columns: Keywords, Gender, Branded_Generic, Topic, Delete_Keep
## Output Example:
## Keywords | Gender | Branded_Generic | Topic | Delete_Keep
## Best shoes | Unisex | Generic | None | Keep
## Best shoes for cycling | Unisex | Generic | cycling | Keep
## Best shoes for running | Unisex | Generic | running | Keep
## Nike shoes | Unisex | Branded | None | Keep
## Nike shoes for cycling | Unisex | Branded | cycling | Keep
## Nike shoes for running | Unisex | Branded | running | Keep
## Adidas shoes | Unisex | Branded | None | Keep
## women's cycling suit | Women | Generic | cycling | Keep


## PART 3: URL Generation
"""

Product - URL	Model - URL	Style - URL	Gender - URL	Sport - URL	Best for - URL	Colour - URL	Features - URL	Collection - URL	Brand - URL	Size - URL	Rise - URL	Sustainable - URL	Material - URL	Teams - URL	Kit Teams - URL	Winter - URL	Outlet - URL	Support - URL	Length - URL	Fit - URL	Surface - URL	Techologies - URL
"""
## those are all the mappings that are available for the URL Generation
## the matching will be done for each of the categories and then concatenated with a custom separator (default is "-")

## There are two parts to this:
## Part 1. Keyword Search Volume & Expansion This is done with the custom scripts we have under KeywordSearchVolume module [DONE]
## Part 2. Keyword Categorization This is done in the aforementioned fashion from the two CSV Files [DONE]
## Part 3. URL Generation - This is done with an extra mapping file that maps the keywords to the URLs

'\n\nProduct - URL\tModel - URL\tStyle - URL\tGender - URL\tSport - URL\tBest for - URL\tColour - URL\tFeatures - URL\tCollection - URL\tBrand - URL\tSize - URL\tRise - URL\tSustainable - URL\tMaterial - URL\tTeams - URL\tKit Teams - URL\tWinter - URL\tOutlet - URL\tSupport - URL\tLength - URL\tFit - URL\tSurface - URL\tTechologies - URL\n'

In [2]:
## reload
%reload_ext autoreload
%autoreload 2

In [3]:
## importing the libraries
import os
import streamlit as st
import pandas as pd
import numpy as np

## custom modules
from KeywordSearchVolume.search_volume_extractor import run_search_volume
from clavis_helpers import (
    does_file_exist,
    parse_search_volume,
    load_excel_and_clean,
    get_payload,
    clean_categories_df,
    categorize_keywords,
    apply_clean_dataframe_for_categorization,
)

In [4]:
## PATHS & PARAMETERS

SAVE_DIR = "./sample_data"

## Part 1. Keyword Search Volume & Expansion
LANGUAGE = "English"  ## this has to be a streamlit input
EXPAND_KEYWORDS = True  ## this has to be a streamlit radio button
GEOLOCATION = "United States"  ## this has to be a streamlit input

## CLAVIS CONFI FILES
CLAVIS_CONFIG_NAME = "Keyword-Categorization-Mapping-Config.xlsx"
CLAVIS_CONFIG_SHEET_NAME = "Config - Categorisation"
CLAVIS_CONFIG_FILE_PATH = os.path.join(SAVE_DIR, CLAVIS_CONFIG_NAME)

## KEYWORDS AND CATEGORY FILES
KEYWORDS_FILE_NAME = "Keywords.csv"
KEYWORDS_FILE_PATH = os.path.join(SAVE_DIR, KEYWORDS_FILE_NAME)

In [5]:
## Part 0. Load the files
catz = pd.read_csv(f"{SAVE_DIR}/categories.csv", sep=";")
KEYWORDS_FILE = pd.read_csv(KEYWORDS_FILE_PATH)

## get the payload
PAYLOAD = get_payload(KEYWORDS_FILE, LANGUAGE, EXPAND_KEYWORDS, GEOLOCATION)


## parse the search volume
if not does_file_exist(f"{SAVE_DIR}/intermediate_results.csv"):
    ## run the search volume
    search_volume = run_search_volume(**PAYLOAD)
    search_volume_df = parse_search_volume(search_volume)
else:
    search_volume_df = pd.read_csv(f"{SAVE_DIR}/intermediate_results.csv")

In [6]:
## load the clavis config file
clavis_config = load_excel_and_clean(CLAVIS_CONFIG_FILE_PATH, CLAVIS_CONFIG_SHEET_NAME)

## function to perform the above steps
separated_categories_dict = clean_categories_df(clavis_config)

## cleaned dataframe
cleaned_categories_df = apply_clean_dataframe_for_categorization(
    separated_categories_dict
)

## separate the URL generation from the categorization
url_generation_df = cleaned_categories_df[
    cleaned_categories_df["Category"].str.contains("URL")
]

## separate the categorization from the URL generation
categorization_df = cleaned_categories_df[
    ~cleaned_categories_df["Category"].str.contains("URL")
]

categorized_keywords = categorize_keywords(search_volume_df, categorization_df)

## now URL generation
generated_urls = categorize_keywords(search_volume_df, url_generation_df)

## get the columns that are not keywords, search volume, idea, or 0
columns_to_concatenate = generated_urls.columns[
    ~generated_urls.columns.isin(["keywords", "search_volume", "idea"])
]

## in all of the columns to concatenate, replace 0 with "", 0 is a str ("0")
generated_urls[columns_to_concatenate] = generated_urls[columns_to_concatenate].replace(
    "0", ""
)

# ## concatenate the columns
generated_urls["url"] = generated_urls[columns_to_concatenate].apply(
    lambda x: "-".join(x.dropna().astype(str)), axis=1
)

## cleaning - remove the hyphen
generated_urls["url"] = generated_urls["url"].replace(
    "-", "", regex=True, inplace=False
)

## replace the ampersand & with ""
generated_urls["url"] = generated_urls["url"].replace(
    "&", "", regex=True, inplace=False
)

## join back the two dataframes
final_df = pd.merge(
    categorized_keywords,
    generated_urls,
    on=["keywords", "search_volume", "idea"],
    how="left",
)

In [10]:
dd = "/Users/eric/Documents/Locaria/Projects/Clavis/allowed_langs_locs/locations_languages.csv"
df = pd.read_csv(dd)

In [33]:
## convert the dataframe entries to a dictionary
lang_loc_dict = df.set_index("location_name").T.to_dict("list")

/var/folders/n5/8tz171ws7vxgbsjsh88bv45h0000gn/T/ipykernel_56831/1360654322.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  lang_loc_dict = df.set_index("location_name").T.to_dict('list')


In [39]:
lang_loc_dict = df.groupby(["location_name"])["language_name"].apply(list).to_dict()

In [42]:
## save the dict as a json file
import json

with open("./allowed_langs_locs/lang_loc_dict.json", "w") as fp:
    json.dump(lang_loc_dict, fp)

In [43]:
from utils import open_json

In [44]:
data = open_json("./allowed_langs_locs/lang_loc_dict.json")

In [49]:
data

{'Algeria': ['French', 'Arabic'],
 'Angola': ['Portuguese'],
 'Argentina': ['Spanish'],
 'Armenia': ['Armenian'],
 'Australia': ['English'],
 'Austria': ['German'],
 'Azerbaijan': ['Azeri'],
 'Bahrain': ['Arabic'],
 'Bangladesh': ['Bengali'],
 'Belgium': ['French', 'Dutch', 'German'],
 'Bolivia': ['Spanish'],
 'Brazil': ['Portuguese'],
 'Bulgaria': ['Bulgarian'],
 'Burkina Faso': ['French'],
 'Cambodia': ['English'],
 'Cameroon': ['French'],
 'Canada': ['English', 'French'],
 'Chile': ['Spanish'],
 'Colombia': ['Spanish'],
 'Costa Rica': ['Spanish'],
 "Cote d'Ivoire": ['French'],
 'Croatia': ['Croatian'],
 'Cyprus': ['Greek', 'English'],
 'Czechia': ['Czech'],
 'Denmark': ['Danish'],
 'Ecuador': ['Spanish'],
 'Egypt': ['Arabic', 'English'],
 'El Salvador': ['Spanish'],
 'Estonia': ['Estonian'],
 'Finland': ['Finnish'],
 'France': ['French'],
 'Germany': ['German'],
 'Ghana': ['English'],
 'Greece': ['Greek'],
 'Guatemala': ['Spanish'],
 'Hong Kong': ['English', 'Chinese (Traditional)']

In [29]:
data["United States"]

'Spanish'